In [5]:
!pip install datasets==1.0.2
!pip install transformers
#!rm seq2seq_trainer.py
#!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!pip install rouge_score

import datasets
import transformers
import pandas as pd
from datasets import Dataset

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from transformers.trainer_seq2seq import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [6]:
import pandas as pd
df=pd.read_csv("keybert_text.csv")[['text', 'summary']]
print("Data size:",len(df))
df.head()

Data size: 152


,text,summary
0,Today is Vasant Panchami. My parents say it i...,The story is about Meenu celebrating Vasant Pa...
1,Alfi was in the garden blowingbubbles.The bubb...,"The text tells the story of Alfi, a boy blowin..."
2,This book belongs toFoxy Joxy Plays A TrickWa...,"The text is a short story about Foxy Joxy, a s..."
3,Madhavpur was a village adjoining a wild life ...,The village of Madhavpur is located next to a ...
4,It’s easy to see animals in the zoo. ...,The text discusses how animals are not just fo...


In [7]:
! pip install datasets

In [8]:
from datasets import Dataset
train_data=Dataset.from_pandas(df[0:61])
val_data=Dataset.from_pandas(df[61:122])
test_data=Dataset.from_pandas(df[122:152])

In [9]:
train_data

Dataset(features: {'text': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, num_rows: 61)

In [10]:
from transformers import RobertaTokenizerFast

In [15]:
!pip uninstall dill
!pip install dill==0.3.5.1

^C


In [17]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
#parameter setting
batch_size=256  #
encoder_max_length=40
decoder_max_length=8

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["text", "summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["text", "summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\MSI\AppData\Local\Temp\ipykernel_10052\3676487541.py", line 27, in <module>
    train_data = train_data.map(
  File "C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\arrow_dataset.py", line 1228, in map
    return self._map_single(
  File "C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\arrow_dataset.py", line 153, in wrapper
    out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)
  File "C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\fingerprint.py", line 157, in wrapper
    kwargs[fingerprint_name] = update_fingerprint(
  File "C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\fingerprint.py", line 105, in 

In [18]:
from transformers import EncoderDecoderModel

roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.10.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.11.crossattention.self.query.weight', 'roberta.encoder.layer.10.crossattention.self

In [19]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 40
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [20]:
import sys
import time
import torch
quantized_model = torch.quantization.quantize_dynamic(
    roberta_shared, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, i

In [21]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(roberta_shared)
print_size_of_model(quantized_model)

Size (MB): 614.781149
Size (MB): 395.20181


In [22]:
!pip install regex requests hydra-core omegaconf bitarray

     ------------------------------------ 154.5/154.5 kB 771.7 kB/s eta 0:00:00
     ---------------------------------------- 79.5/79.5 kB 1.5 MB/s eta 0:00:00
     ------------------------------------ 118.2/118.2 kB 631.4 kB/s eta 0:00:00
     -------------------------------------- 117.0/117.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144578 sha256=bf4ed024ff8a1f9744d5e2f492a360552e1a5c9e87d9af96f72fd3e136b8753d
  Stored in directory: c:\users\msi\appdata\local\pip\cache\wheels\12\93\dd\1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [23]:
!pip install --upgrade fairseq

     ---------------------------------------- 9.6/9.6 MB 373.2 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Cython-0.29.34-py2.py3-none-any.whl (988 kB)
     -------------------------------------- 123.8/123.8 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 118.9/118.9 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 1.1 MB/s eta 0:00:00
     ------------------------------------ 112.4/112.4 kB 817.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cr

In [24]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

Downloading: "https://github.com/pytorch/fairseq/zipball/main" to C:\Users\MSI/.cache\torch\hub\main.zip
2023-05-01 22:33:52 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data\data_utils_fast.cpp


C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\cpp_extension.py:359: UserWarning: Error checking compiler version for cl: [WinError 2] Le fichier spécifié est introuvable
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data\token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build\temp.win-amd64-cpython-310
creating build\temp.win-amd64-cpython-310\Release
creating build\temp.win-amd64-cpython-310\Release\fairseq
creating build\temp.win-amd64-cpython-310\Release\fairseq\clib
creating build\temp.win-amd64-cpython-310\Release\fairseq\clib\libbleu
"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.34.31933\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\include -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\Include "-IC:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.34.31933\include" "-IC:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.22000.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\\include\10.0

building 'fairseq.libbase' extension
creating build\temp.win-amd64-cpython-310\Release\fairseq\clib\libbase
"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.34.31933\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\include -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\include\torch\csrc\api\include -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\include\TH -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\include\THC -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\include -IC:\Users\MSI\AppData\Local\Programs\Python\Python310\Include "-IC:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.34.31933\include" "-IC:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.22000.0\ucrt

copying build\lib.win-amd64-cpython-310\fairseq\libbleu.cp310-win_amd64.pyd -> fairseq
copying build\lib.win-amd64-cpython-310\fairseq\data\data_utils_fast.cp310-win_amd64.pyd -> fairseq\data
copying build\lib.win-amd64-cpython-310\fairseq\data\token_block_utils_fast.cp310-win_amd64.pyd -> fairseq\data
copying build\lib.win-amd64-cpython-310\fairseq\libbase.cp310-win_amd64.pyd -> fairseq
copying build\lib.win-amd64-cpython-310\fairseq\libnat.cp310-win_amd64.pyd -> fairseq
copying build\lib.win-amd64-cpython-310\alignment_train_cpu_binding.cp310-win_amd64.pyd -> 


2023-05-01 22:34:36 | INFO | fairseq.file_utils | http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz not found in cache, downloading to C:\Users\MSI\AppData\Local\Temp\tmpcuyal5yu
100%|█████████████████████████████████████████████████████████████████| 655283069/655283069 [19:42<00:00, 554224.34B/s]
2023-05-01 22:54:18 | INFO | fairseq.file_utils | copying C:\Users\MSI\AppData\Local\Temp\tmpcuyal5yu to cache at C:\Users\MSI\.cache\torch\pytorch_fairseq\83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
2023-05-01 22:54:19 | INFO | fairseq.file_utils | creating metadata file for C:\Users\MSI\.cache\torch\pytorch_fairseq\83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
2023-05-01 22:54:19 | INFO | fairseq.file_utils | removing temp file C:\Users\MSI\AppData\Local\Temp\tmpcuyal5yu
2023-05-01 22:54:19 | INFO | fairseq.

2023-05-01 22:54:35 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json not found in cache, downloading to C:\Users\MSI\AppData\Local\Temp\tmpce0rn_bo
1042301B [00:02, 433327.42B/s]
2023-05-01 22:54:40 | INFO | fairseq.file_utils | copying C:\Users\MSI\AppData\Local\Temp\tmpce0rn_bo to cache at C:\Users\MSI\.cache\torch\pytorch_fairseq\e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2023-05-01 22:54:40 | INFO | fairseq.file_utils | creating metadata file for C:\Users\MSI\.cache\torch\pytorch_fairseq\e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2023-05-01 22:54:40 | INFO | fairseq.file_utils | removing temp file C:\Users\MSI\AppData\Local\Temp\tmpce0rn_bo
2023-05-01 22:54:43 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe not found in cache, dow

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-23): 24 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

In [25]:
import torch.nn.utils.prune as prune
for name, module in roberta_shared.named_modules():
    if isinstance(module, torch.nn.Linear):
       pruned_model= prune.random_unstructured(module, name='weight', amount=0.5)
       print_size_of_model(pruned_model)
#pruned_model=prune.random_unstructured(roberta_shared, name='weight',amount=0.5)


Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 4.722772
Size (MB): 18.887764
Size (MB): 18.878548
Size (MB): 4.722772
Size

In [26]:
!pip install rouge_score

In [27]:
from datasets import  load_metric
# load rouge for validation
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [28]:
from transformers import Trainer


In [29]:
training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    do_train=True,
    do_eval=True,
    logging_steps=2, 
    save_steps=16, 
    eval_steps=500, 
    warmup_steps=500, 
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=False, 
    
)
#predict_with_generate=True,

# instantiate trainer
trainer = Trainer(
     model=roberta_shared,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)
trainer.train()

C:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,encoder_outputs,past_key_values,inputs_embeds,decoder_inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,kwargs,labels,label,label_ids.

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = EncoderDecoderModel.from_pretrained("/checkpoint-6432")
model.to("cuda")
batch_size = 1024

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=40, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]